<a href="https://colab.research.google.com/github/Stepka/Bayesian-NN-over-Amazon-Games-Rates/blob/master/Bayesian_NN_over_Amazon_Games_Rates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install the latest Tensorflow version.
# !pip3 uninstall tensorflow
# !pip3 uninstall tensorflow-probability

In [0]:
# import tensorflow as tf
# import tensorflow_probability as tfp

# print("TensorFlow version:", tf.__version__)
# print("TensorFlow probability version:", tfp.__version__)
# print("Eager mode:", tf.executing_eagerly())

In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet tensorflow==1.13.1
!pip3 install --quiet --upgrade tensorflow-probability
!pip3 install --quiet tensorflow-hub

In [4]:
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_hub as hub

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

W0409 16:18:34.293462 140269118805888 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [5]:


print("TensorFlow version:", tf.__version__)
print("TensorFlow probability version:", tfp.__version__)
print("Pandas version:", pd.__version__)
tf.executing_eagerly()

TensorFlow version: 1.13.1
TensorFlow probability version: 0.6.0
Pandas version: 0.22.0


False

In [6]:
from google.colab import drive
drive.mount('/drive')

default_path = "/drive/My Drive/Colab Notebooks/Bayesian Amazon Video Rates/data/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive


In [0]:
######################
###   load data    ###
######################

In [0]:
dataset_json = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz", lines=True)

In [9]:

print("Num rows: {}\n".format(len(dataset_json.index)))

dataset_json.head(10)

Num rows: 231780



,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0700099867,"[8, 12]",1,Installing the game was a struggle (because of...,"07 9, 2012",A2HD75EMZR8QLN,123,Pay to unlock content? I don't think so.,1341792000
1,0700099867,"[0, 0]",4,If you like rally cars get this game you will ...,"06 30, 2013",A3UR8NLLY1ZHCX,"Alejandro Henao ""Electronic Junky""",Good rally game,1372550400
2,0700099867,"[0, 0]",1,1st shipment received a book instead of the ga...,"06 28, 2014",A1INA0F5CWW3J4,"Amazon Shopper ""Mr.Repsol""",Wrong key,1403913600
3,0700099867,"[7, 10]",3,"I got this version instead of the PS3 version,...","09 14, 2011",A1DLMTOTHQ4AST,ampgreen,"awesome game, if it did not crash frequently !!",1315958400
4,0700099867,"[2, 2]",4,I had Dirt 2 on Xbox 360 and it was an okay ga...,"06 14, 2011",A361M14PU2GUEG,"Angry Ryan ""Ryan A. Forrest""",DIRT 3,1308009600
5,0700099867,"[0, 0]",4,"Overall this is a well done racing game, with ...","05 11, 2013",A2UTRVO4FDCBH6,A.R.G.,"Good racing game, terrible Windows Live Requir...",1368230400
6,0700099867,"[11, 13]",5,Loved playing Dirt 2 and I thought the graphic...,"08 14, 2011",AN3YYDZAS3O1Y,Bob,A step up from Dirt 2 and that is terrific!,1313280000
7,0700099867,"[1, 4]",1,I can't tell you what a piece of dog**** this ...,"11 24, 2012",AQTC623NCESZW,Chesty Puller,Crash 3 is correct name AKA Microsoft,1353715200
8,0700099867,"[0, 1]",4,I initially gave this one star because it was ...,"11 14, 2012",A1QJJU33VNC4S7,D@rkFX,A great game ruined by Microsoft's account man...,1352851200
9,0700099867,"[1, 1]",2,I still haven't figured this one out. Did ever...,"02 8, 2014",A2JLT2WY0F2HVI,D. Sweetapple,Couldn't get this one to work,1391817600


In [0]:
dataset_json.drop(labels = ["asin", "helpful", "reviewText",  "reviewTime", "reviewerID", "reviewerName", "unixReviewTime"], axis = 1, inplace = True)

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3", "https://tfhub.dev/google/nnlm-en-dim128/1"]

In [12]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

Instructions for updating:
Colocations handled automatically by placer.


W0409 16:20:22.724450 140269118805888 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:

  
def extract_embedings(messages, name, use_cache=True):

  # Reduce logging output.
  tf.logging.set_verbosity(tf.logging.ERROR)

  print("Messages  length: {}\n".format(len(messages)))

  EMBEDINGS_BATCH_SIZE = 10000

  message_embeddings = []
  start_step = 0

  if use_cache:
    try:
      message_embeddings_df = pd.read_csv(default_path + name + '_embedings.csv')
    except FileNotFoundError:
      print("no saved embedings")
      message_embeddings_df = None

    if message_embeddings_df is not None: 
      print("we have", len(message_embeddings_df.index), "saved embedings")
      start_step = int(len(message_embeddings_df.index) / EMBEDINGS_BATCH_SIZE)
      print("will start from", start_step, "step")
      message_embeddings = message_embeddings_df.values

  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])

    num_steps = int( 1 + len(messages) / EMBEDINGS_BATCH_SIZE )
    print("Num steps: {}\n".format(num_steps))
    if start_step < num_steps and len(message_embeddings) < len(messages):
      for step in range(start_step, num_steps):
        embeddings = session.run(embed(messages[EMBEDINGS_BATCH_SIZE*step : EMBEDINGS_BATCH_SIZE*(step+1)]))
    #     message_embeddings = message_embeddings + embeddings
        if len(message_embeddings) == 0:
          message_embeddings = embeddings
        else :
          message_embeddings = np.vstack((message_embeddings, embeddings))

        message_embeddings_df = pd.DataFrame(data = message_embeddings)
  #       pd.DataFrame(message_embeddings_df).to_csv(default_path + name + '_embedings.csv', index=False, header=True)

  #       if step % 10 == 0:
        pd.DataFrame(message_embeddings_df).to_csv(default_path + name + '_embedings ' + str(step) + '.csv', index=False, header=True)

        print("Step {}, Embedding size: {}".format(step, len(message_embeddings)))
      
        if len(message_embeddings) == len(messages):
          pd.DataFrame(message_embeddings_df).to_csv(default_path + name + '_embedings.csv', index=False, header=True)
          
    else:
      print("We have all data")
      
          
      
  return message_embeddings, message_embeddings_df

In [14]:
embedings, embedings_df = extract_embedings(dataset_json["summary"].values, "summary")

Messages  length: 231780

we have 231780 saved embedings
will start from 23 step
Num steps: 24

We have all data


In [0]:
# try:
#   df = pd.read_csv(default_path + 'summary_embedings.csv')
# except FileNotFoundError:
#   print("no file")
#   df = None

# if df is not None: 
#   print(len(df.index))
  
# df.head()

In [16]:
dataset = pd.concat([dataset_json, embedings_df], axis=1)

dataset.head()

,overall,summary,0,1,2,3,4,5,6,7,...,502,503,504,505,506,507,508,509,510,511
0,1,Pay to unlock content? I don't think so.,0.046938,0.042071,0.024881,0.045308,-0.019535,0.081542,0.083805,0.071415,...,0.010543,0.037760,-0.004364,-0.044930,-0.047975,-0.018246,-0.048304,-0.070092,-0.066281,0.050676
1,4,Good rally game,0.016563,-0.031776,-0.043782,-0.042133,0.021630,-0.033483,0.011921,0.006047,...,-0.031585,0.049901,-0.031180,-0.048357,-0.077174,0.016070,0.042710,-0.037952,-0.028943,0.030393
2,1,Wrong key,0.019536,-0.011373,-0.010637,0.014312,-0.035867,-0.022187,-0.002389,-0.061504,...,-0.004081,-0.001812,-0.025526,-0.063241,-0.036931,0.018444,0.039002,0.050907,-0.056628,-0.006743
3,3,"awesome game, if it did not crash frequently !!",0.041646,-0.020160,0.010884,0.035234,-0.005660,-0.006740,0.054527,0.017031,...,-0.030309,-0.013751,-0.054002,0.061169,-0.031388,-0.077839,-0.043388,-0.047516,0.072083,0.038782
4,4,DIRT 3,0.023018,-0.029862,-0.074362,-0.023005,0.023212,0.066392,0.032889,0.023983,...,-0.044928,-0.003295,0.002224,-0.032011,-0.064896,-0.062261,0.027037,-0.014742,0.011601,0.090250


In [17]:
summaries = dataset["summary"]

print(dataset['overall'][9000], summaries[9000])
print(dataset['overall'][9001], summaries[9001])
print(dataset['overall'][9002], summaries[9002])
print(dataset['overall'][9003], summaries[9003])
print(dataset['overall'][9004], summaries[9004])
print(dataset['overall'][9005], summaries[9005])

2 A game to rent, but not to buy
3 Fun if you love pokemon.
2 A big thumbs down
4 a pretty cool game
5 The #1 arcade smash is back!  Upright funny!
4 Fun for all


In [0]:
dataset.drop(labels = ["summary"], axis = 1, inplace = True)

In [19]:
embeding_columns = list(dataset)
score_column = 'overall'
embeding_columns.remove(score_column)

print("features: ", embeding_columns)
print("target feature: ", score_column)

features:  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156',

In [0]:
# Copyright 2018 The TensorFlow Probability Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ============================================================================
"""Trains a Bayesian neural network to classify MNIST digits.
The architecture is LeNet-5 [1].
#### References
[1]: Yann LeCun, Leon Bottou, Yoshua Bengio, and Patrick Haffner.
     Gradient-based learning applied to document recognition.
     _Proceedings of the IEEE_, 1998.
     http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import warnings

# Dependency imports
from absl import flags
import matplotlib
matplotlib.use("Agg")
from matplotlib import figure  # pylint: disable=g-import-not-at-top
from matplotlib.backends import backend_agg
import matplotlib.patches as patches
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

from tensorflow.contrib.learn.python.learn.datasets import mnist

tfd = tfp.distributions

IMAGE_SHAPE = [28, 28, 1]

####Delete all flags before declare#####

# def del_all_flags(FLAGS):
#     flags_dict = FLAGS._flags()    
#     keys_list = [keys for keys in flags_dict]    
#     for keys in keys_list:
#         FLAGS.__delattr__(keys)

# del_all_flags(tf.flags.FLAGS)

LEARNING_RATE=0.001
MAX_STEPS=6000
BATCH_SIZE=128
DATA_DIR=os.path.join(os.getenv("CONTENT", "/content"), "bayesian_amazon/data")
MODEL_DIR=os.path.join(os.getenv("CONTENT", "/content"), "bayesian_amazon/")
VIZ_STEPS=400
NUM_MONTE_CARLO=50
VALIDATION_SIZE = 10000


def plot_weight_posteriors(names, qm_vals, qs_vals, fname):
  """Save a PNG plot with histograms of weight means and stddevs.
  Args:
    names: A Python `iterable` of `str` variable names.
    qm_vals: A Python `iterable`, the same length as `names`,
      whose elements are Numpy `array`s, of any shape, containing
      posterior means of weight varibles.
    qs_vals: A Python `iterable`, the same length as `names`,
      whose elements are Numpy `array`s, of any shape, containing
      posterior standard deviations of weight varibles.
    fname: Python `str` filename to save the plot to.
  """
  fig = figure.Figure(figsize=(6, 3))
  canvas = backend_agg.FigureCanvasAgg(fig)

  ax = fig.add_subplot(1, 2, 1)
  for n, qm in zip(names, qm_vals):
    sns.distplot(qm.flatten(), ax=ax, label=n)
  ax.set_title("weight means")
  ax.set_xlim([-1.5, 1.5])
  ax.legend()

  ax = fig.add_subplot(1, 2, 2)
  for n, qs in zip(names, qs_vals):
    sns.distplot(qs.flatten(), ax=ax)
  ax.set_title("weight stddevs")
  ax.set_xlim([0, 1.])

  fig.tight_layout()
  canvas.print_figure(fname, format="png")
  print("saved {}".format(fname))


def plot_heldout_prediction(start_row, probs,
                            fname, n=10, title=""):
  """Save a PNG plot visualizing posterior uncertainty on heldout data.
  Args:
    input_vals: 
    probs: A `float`-like Numpy array of shape `[num_monte_carlo,
      num_heldout, num_classes]` containing Monte Carlo samples of
      class probabilities for each heldout sample.
    fname: Python `str` filename to save the plot to.
    n: Python `int` number of datapoints to vizualize.
    title: Python `str` title for the plot.
  """
  fig = figure.Figure(figsize=(9, 3*n))
  canvas = backend_agg.FigureCanvasAgg(fig)
  

  for i in range(n):
    ax = fig.add_subplot(n, 3, 3*i + 1)
#     ax.imshow(input_vals[i, :].reshape(IMAGE_SHAPE[:-1]), interpolation="None")
    # axes coordinates are 0,0 is bottom left and 1,1 is upper right
    ax.text(0.5, 0.5, 
            summaries[start_row + i],
            horizontalalignment='center',
            verticalalignment='center', 
            wrap=True,
            fontsize=10,
            transform=ax.transAxes)

    ax = fig.add_subplot(n, 3, 3*i + 2)
    for prob_sample in probs:
      sns.barplot(np.arange(6), prob_sample[i, :], alpha=0.1, ax=ax)
      ax.set_ylim([0, 1])
    ax.set_title("posterior samples")

    ax = fig.add_subplot(n, 3, 3*i + 3)
    sns.barplot(np.arange(6), np.mean(probs[:, i, :], axis=0), ax=ax)
    ax.set_ylim([0, 1])
    ax.set_title("predictive probs")
  fig.suptitle(title)
  fig.tight_layout()

  canvas.print_figure(fname, format="png")
  print("saved {}".format(fname))



In [0]:


def build_input_pipeline(df, features, target_feature, batch_size, heldout_size):
  """Build an Iterator switching between train and heldout data."""

  train_X = df[features][heldout_size:]
  train_y = df[target_feature][heldout_size:]
  
  validation_X = df[features][:heldout_size]
  validation_y = df[target_feature][:heldout_size]
  
  # Build an iterator over training batches.
  training_dataset = tf.data.Dataset.from_tensor_slices(
      (
      tf.cast(train_X.values, tf.float32), 
      tf.cast(train_y.values, tf.int32)
      )
  )
  training_batches = training_dataset.shuffle(
      50000, reshuffle_each_iteration=True).repeat().batch(batch_size)
  training_iterator = tf.compat.v1.data.make_one_shot_iterator(training_batches)

  # Build a iterator over the heldout set with batch_size=heldout_size,
  # i.e., return the entire heldout set as a constant.
  heldout_dataset = tf.data.Dataset.from_tensor_slices(
      (
      tf.cast(validation_X.values, tf.float32), 
      tf.cast(validation_y.values, tf.int32)
      )
  )
  heldout_frozen = (heldout_dataset.take(heldout_size).
                    repeat().batch(heldout_size))
  heldout_iterator = tf.compat.v1.data.make_one_shot_iterator(heldout_frozen)

  # Combine these into a feedable iterator that can switch between training
  # and validation inputs.
  handle = tf.compat.v1.placeholder(tf.string, shape=[])
  feedable_iterator = tf.compat.v1.data.Iterator.from_string_handle(
      handle, training_batches.output_types, training_batches.output_shapes)
  feature_sets, labels = feedable_iterator.get_next()

  return feature_sets, labels, handle, training_iterator, heldout_iterator


def main(train_df, features, target_feature):
#   del argv  # unused
  if tf.io.gfile.exists(MODEL_DIR):
    tf.compat.v1.logging.warning(
        "Warning: deleting old log directory at {}".format(MODEL_DIR))
    tf.io.gfile.rmtree(MODEL_DIR)
  tf.io.gfile.makedirs(MODEL_DIR)

  ############################################################################
  
#   mnist_data = mnist.read_data_sets(FLAGS.data_dir, reshape=False)

  (embedings, labels, handle,
   training_iterator, heldout_iterator) = build_input_pipeline(
       train_df, features, target_feature, 
      BATCH_SIZE, VALIDATION_SIZE)

  ############################################################################
  # Build a Bayesian LeNet5 network. We use the Flipout Monte Carlo estimator
  # for the convolution and fully-connected layers: this enables lower
  # variance stochastic gradients than naive reparameterization.
  with tf.compat.v1.name_scope("bayesian_neural_net", values=[embedings]):
    neural_net = tf.keras.Sequential([
        tfp.layers.DenseFlipout(512, activation=tf.nn.relu),
        tfp.layers.DenseFlipout(6)
        ])

    logits = neural_net(embedings)
    labels_distribution = tfd.Categorical(logits=logits)
    log_probs = labels_distribution.log_prob(labels)


  
  #############################################################################
  

  # Compute the -ELBO as the loss, averaged over the batch size.
  neg_log_likelihood = -tf.reduce_mean(
      input_tensor=labels_distribution.log_prob(labels))
  kl = sum(neural_net.losses) / (len(train_df.index) - VALIDATION_SIZE)
  elbo_loss = neg_log_likelihood + kl

  # Build metrics for evaluation. Predictions are formed from a single forward
  # pass of the probabilistic layers. They are cheap but noisy predictions.
  predictions = tf.argmax(input=logits, axis=1)
  accuracy, accuracy_update_op = tf.compat.v1.metrics.accuracy(
      labels=labels, predictions=predictions)

  # Extract weight posterior statistics for layers with weight distributions
  # for later visualization.
  names = []
  qmeans = []
  qstds = []
  for i, layer in enumerate(neural_net.layers):
    try:
      q = layer.kernel_posterior
    except AttributeError:
      continue
    names.append("Layer {}".format(i))
    qmeans.append(q.mean())
    qstds.append(q.stddev())

  with tf.compat.v1.name_scope("train"):
    optimizer = tf.compat.v1.train.AdamOptimizer(
        learning_rate=LEARNING_RATE)
    train_op = optimizer.minimize(elbo_loss)

  init_op = tf.group(tf.compat.v1.global_variables_initializer(),
                     tf.compat.v1.local_variables_initializer())
  
###########################################################################
  
  with tf.compat.v1.Session() as sess:
    sess.run(init_op)

    # Run the training loop.
    train_handle = sess.run(training_iterator.string_handle())
    heldout_handle = sess.run(heldout_iterator.string_handle())
    for step in range(MAX_STEPS):
      _ = sess.run([train_op, accuracy_update_op],
                   feed_dict={handle: train_handle})

      if step % 100 == 0:
        loss_value, accuracy_value = sess.run(
            [elbo_loss, accuracy], feed_dict={handle: train_handle})
        print("Step: {:>3d} Loss: {:.3f} Accuracy: {:.3f}".format(
            step, loss_value, accuracy_value))

      if (step+1) % VIZ_STEPS == 0:
        # Compute log prob of heldout set by averaging draws from the model:
        # p(heldout | train) = int_model p(heldout|model) p(model|train)
        #                   ~= 1/n * sum_{i=1}^n p(heldout | model_i)
        # where model_i is a draw from the posterior p(model|train).
        probs = np.asarray([sess.run((labels_distribution.probs),
                                     feed_dict={handle: heldout_handle})
                            for _ in range(NUM_MONTE_CARLO)])
        mean_probs = np.mean(probs, axis=0)

        embedings_vals, label_vals = sess.run((embedings, labels),
                                          feed_dict={handle: heldout_handle})
        heldout_lp = np.mean(np.log(mean_probs[np.arange(mean_probs.shape[0]),
                                               label_vals.flatten()]))
        print(" ... Held-out nats: {:.3f}".format(heldout_lp))

        qm_vals, qs_vals = sess.run((qmeans, qstds))

        plot_weight_posteriors(names, qm_vals, qs_vals,
                               fname=os.path.join(
                                   MODEL_DIR,
                                   "step{:05d}_weights.png".format(step)))

        plot_heldout_prediction(len(train_df.index) - VALIDATION_SIZE, probs,
                                fname=os.path.join(
                                    MODEL_DIR,
                                    "step{:05d}_pred.png".format(step)),
                                title="mean heldout logprob {:.2f}"
                                .format(heldout_lp))
# if __name__ == "__main__":
#   tf.compat.v1.app.run()

In [22]:
main(dataset[:150000], embeding_columns, score_column)

Step:   0 Loss: 6.477 Accuracy: 0.109
Step: 100 Loss: 5.683 Accuracy: 0.527
Step: 200 Loss: 5.490 Accuracy: 0.546
Step: 300 Loss: 5.355 Accuracy: 0.553
 ... Held-out nats: -0.903


/usr/local/lib/python3.6/dist-packages/matplotlib/axes/_axes.py:6521: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  alternative="'density'", removal="3.1")


saved /content/bayesian_amazon/step00399_weights.png


/usr/local/lib/python3.6/dist-packages/seaborn/categorical.py:1428: FutureWarning: remove_na is deprecated and is a private function. Do not use.
  stat_data = remove_na(group_data)


saved /content/bayesian_amazon/step00399_pred.png
Step: 400 Loss: 5.390 Accuracy: 0.557
Step: 500 Loss: 5.277 Accuracy: 0.562
Step: 600 Loss: 5.094 Accuracy: 0.564
Step: 700 Loss: 4.909 Accuracy: 0.568
 ... Held-out nats: -0.905
saved /content/bayesian_amazon/step00799_weights.png
saved /content/bayesian_amazon/step00799_pred.png
Step: 800 Loss: 5.038 Accuracy: 0.570
Step: 900 Loss: 4.926 Accuracy: 0.571
Step: 1000 Loss: 4.637 Accuracy: 0.573
Step: 1100 Loss: 4.777 Accuracy: 0.574
 ... Held-out nats: -0.889
saved /content/bayesian_amazon/step01199_weights.png
saved /content/bayesian_amazon/step01199_pred.png
Step: 1200 Loss: 4.520 Accuracy: 0.575
Step: 1300 Loss: 4.549 Accuracy: 0.576
Step: 1400 Loss: 4.421 Accuracy: 0.576
Step: 1500 Loss: 4.430 Accuracy: 0.577
 ... Held-out nats: -0.894
saved /content/bayesian_amazon/step01599_weights.png
saved /content/bayesian_amazon/step01599_pred.png
Step: 1600 Loss: 4.300 Accuracy: 0.577
Step: 1700 Loss: 4.155 Accuracy: 0.577
Step: 1800 Loss: 4.1